In [52]:
from bs4 import BeautifulSoup
import requests as req
import pandas as pd
import numpy as np
import re


## Get map list from game res and map type from wotreplays.ru

In [3]:
with open("D:\\Games\\World_of_Tanks\\res\\text\lc_messages\\arenas.mo", "rb") as f:
    arenas = f.read()


In [20]:
work_names_start = arenas.find(b"01_karelia")
work_names_end = arenas.find(b"Project-Id-Version: World Of Tanks") - 1

desc_start = arenas.find(b"Content-Transfer-Encoding: 8bit\n") + \
    len("Content-Transfer-Encoding: 8bit\n") + 1
desc_end= len(arenas) - 1


In [21]:
work_names = [i.decode("utf8") for i in arenas[work_names_start:work_names_end].split(b"\x00")]
desc = [i.decode("utf8") for i in arenas[desc_start:desc_end].split(b"\x00")]


In [67]:
maps = []
for wn, d in zip(work_names, desc):
    if re.match(r"^\d+_.*name$", wn):
        id_map = int(wn.split("_")[0])
        is_standart = len(BeautifulSoup(req.get(
            f"http://wotreplays.ru/site/index/battle_type/1/map/{id_map}").text, "html.parser") \
            .find_all(class_="btn_l-grey")[:-1]) != 0
        try:
            resp = req.get("https://wiki.wargaming.net/ru/Tank:"+d.replace(" ", "_")).text
            pos = resp.find("×")
            size = int(resp[pos+1:pos+4])
            if size < 500:
                size *= 10
        except:
            print("Not found:", d)
            size = np.nan
        maps.append({
            "id": id_map,
            "work_name": wn[:-5],
            "name": d,
            "standart": is_standart,
            "map_size": size
        })
maps = pd.DataFrame(maps).sort_values("id")


Not found: Каньон
Not found: Волга
Not found: Воздушная гавань
Not found: Устричный залив
Not found: Ледник
Not found: Дальний Восток
Not found: Предгорья
Not found: Место силы
Not found: Застава


In [68]:
maps

,id,work_name,name,standart,map_size
0,1,01_karelia,Карелия,True,1000.0
1,2,02_malinovka,Малиновка,True,1000.0
2,3,03_campania_big,Провинция,True,850.0
3,4,04_himmelsdorf,Химмельсдорф,True,700.0
4,5,05_prohorovka,Прохоровка,True,1000.0
5,6,06_ensk,Энск,True,600.0
6,7,07_lakeville,Ласвилль,True,800.0
7,8,08_ruinberg,Руинберг,True,850.0
10,10,10_hills,Рудники,True,800.0
16,11,11_murovanka,Мурованка,True,1000.0


In [69]:
maps.to_csv("../data/assets/maps.csv", index=False)